In [3]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


In [4]:
famago = pd.read_csv('../data/private/csv/fam/august.csv')
famsep = pd.read_csv('../data/private/csv/fam/september.csv')
famoct = pd.read_csv('../data/private/csv/fam/october.csv')

ago = famago[famago['day'] >= 29]
oct = famoct[famoct['day'] <= 7]
famdf = pd.concat([ago, famsep, oct], sort=False)
famdf.head()

,time,sysclass,lat,lon,dir,vel,size,ttyyyxx1,ttyyyxx2,ttyyyxx3,...,maxz,meanvil,ttvil,meanprec,maxprec,year,month,day,hour,minute
13486,0.0,0.0,-2.57,-58.57,-999.0,0.0,49.0,0.0,0.0,0.0,...,25.35,0.25,12.01,1.18,2.72,2014,8,29,0,24
13487,0.2,1.0,-2.47,-58.63,329.0,64.8,23.0,0.0,0.0,0.0,...,23.67,0.21,4.80,1.08,2.11,2014,8,29,0,24
13488,0.4,1.0,-2.39,-58.67,333.0,49.7,26.0,0.0,0.0,0.0,...,23.67,0.22,5.61,1.21,2.09,2014,8,29,0,24
13489,0.6,1.0,-2.29,-58.71,338.0,59.8,32.0,0.0,0.0,0.0,...,27.83,0.26,8.24,1.53,3.81,2014,8,29,0,24
13490,0.8,1.0,-2.29,-58.77,270.0,33.3,13.0,0.0,0.0,0.0,...,24.28,0.23,2.94,1.41,2.16,2014,8,29,0,24


In [5]:
no = famdf[famdf['ttyyyxx3'] == 0]
yes = famdf[famdf['ttyyyxx3'] > 0]
yes.head()

,time,sysclass,lat,lon,dir,vel,size,ttyyyxx1,ttyyyxx2,ttyyyxx3,...,maxz,meanvil,ttvil,meanprec,maxprec,year,month,day,hour,minute
6,0.6,1.0,-3.55,-58.53,0.0,44.4,94.0,0.0,0.0,2.0,...,49.32,1.67,156.71,11.71,113.17,2014,9,1,16,24
8,1.0,1.0,-3.53,-58.61,296.0,24.8,114.0,1.0,1.0,5.0,...,44.58,1.16,131.70,7.03,53.78,2014,9,1,16,24
11,1.6,1.0,-3.51,-58.81,243.0,24.8,114.0,0.0,1.0,1.0,...,46.06,1.19,136.06,6.83,68.02,2014,9,1,16,24
13,2.0,1.0,-3.47,-59.01,303.0,40.0,210.0,3.0,2.0,3.0,...,45.78,1.53,321.40,7.98,64.93,2014,9,1,16,24
14,2.2,1.0,-3.47,-59.05,270.0,22.2,199.0,1.0,1.0,7.0,...,46.71,1.75,348.60,9.58,75.20,2014,9,1,16,24


In [6]:
fig = go.Figure(layout=go.Layout(
    title='Comparação da distribuição do índice maxz entre subconjuntos com e sem raios'))

fig.add_trace(
    go.Box(y=no['maxz'], #boxpoints='all',
    name='Subconjunto sem descargas atmosféricas',
    marker_color='#708D81', boxmean='sd')
)

fig.add_trace(
    go.Box(y=yes['maxz'], #boxpoints='all',
    name='Subconjunto com descargas atmosféricas',
    marker_color='#8D0801')
)

# fig.update_traces(textposition='top center')

fig.update_layout(
#     template='plotly_dark',
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Máxima refletividade (dBZ)"
        )
    )
)

iplot(fig)

In [13]:
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
thresholds = [0, 10, 30, 100]
# indexs = [(1, 1), (1, 2), (2, 1), (2, 2)]
indexs = [(1, 1), (2, 1), (3, 1), (4, 1)]
colors = [
    ('#8C8C8C', '#F18F01'), ('#4C333E', '#0053A0'),
    ('#AF4F13', '#3EAD4F'), ('#708D81', '#8D0801')
]

for index, threshold, color in zip(indexs, thresholds, colors):
    row, line = index
    cset1, cset2 = color
    no = famdf[famdf['ttyyyxx3'] <= threshold]
    yes = famdf[famdf['ttyyyxx3'] > threshold]
    
    fig.add_trace (
        go.Box(x=no['maxz'], 
        name='Subconjunto com %s ou menos raios detectados'%threshold,
        marker_color=cset1),
        row=row, col=line,
    )

    fig.add_trace (
        go.Box(x=yes['maxz'], 
        name='Subconjunto com mais de %s raios detectados'%threshold,
        marker_color=cset2),
        row=row, col=line
    )
    
    
    fig.update_yaxes(
        showticklabels=False
    )
 
fig['layout']['xaxis4'].update(title='Máxima refletividade (dBZ)')

fig.update_layout(
    legend=dict(x=0, y=-0.3),
    legend_orientation="h",
    font={'size':14},
    title='Comparação da distribuição do índice <b>maxz</b> entre <br />'\
    'subconjuntos com e sem descargas elétricas atmosféricas', template='plotly_dark',
)

iplot(fig)